## Demo 1: Bivairate Gaussian Distribution

Today, we will learn some intuition about the bivariate Gaussian.

### Goals:
1. Get intuition about how the Gaussian distribution is affected by its covariance matrix and mean vector.
2. We may find an orthogonal matrix (like a rotational operator) to de-correlate the two variables.
3. Covariance matrix is positive definite, which has nice properties.

### Highlights:
A part of the DEMO today is based on Christian Hill's [Visualizing the bivariate Gaussian distribution](https://scipython.com/blog/visualizing-the-bivariate-gaussian-distribution/).

The [Einstein notation](https://en.wikipedia.org/wiki/Einstein_notation) is a nice way to represent a complicated matrix multiplication. It's realized in NumPy's `einsum()` function. See [Manual](https://numpy.org/doc/stable/reference/generated/numpy.einsum.html) or [this post](https://ajcr.net/Basic-guide-to-einsum/).

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from matplotlib import cm
from mpl_toolkits.mplot3d import Axes3D

# Our 2-dimensional distribution will be over variables X and Y
N = 50
X = np.linspace(-3, 3, N)
Y = np.linspace(-3, 4, N)
X, Y = np.meshgrid(X, Y)

# Mean vector and covariance matrix
mu = np.array([0.0, 1.0])
sigma = np.array([[1.0, -0.5], [-0.5, 1.5]])

# Pack X and Y into a single 3-dimensional array
pos = np.empty(X.shape + (2,))
pos[:, :, 0] = X
pos[:, :, 1] = Y


def multivariate_gaussian(pos, mu, sigma):
    """Return the multivariate Gaussian distribution on array positions.

    pos is an array constructed by packing the meshed arrays of variables
    x_1, x_2, x_3, ..., x_k into its _last_ dimension.

    """

    n = mu.shape[0]
    sigma_det = np.linalg.det(sigma)
    sigma_inv = np.linalg.inv(sigma)
    N = np.sqrt((2 * np.pi) ** n * sigma_det)
    
    # This einsum call calculates (x-mu)T.sigma-1.(x-mu) in a vectorized
    # way across all the input variables.
    fac = np.einsum("ijk,kl,ijl->ij", pos - mu, sigma_inv, pos - mu)

    return np.exp(-fac / 2) / N


# The distribution on the variables X, Y packed into pos.
Z = multivariate_gaussian(pos, mu, sigma)

In [ ]:
# We can visualize the covariance matrix...
import seaborn as sns
ax = sns.heatmap(sigma, annot=True, linewidth=1, cmap="viridis")
ax.set(xlabel="x", ylabel="y")
ax.xaxis.tick_top()

In [ ]:
# Let's plot the 2D probability density function!
ax = sns.heatmap(Z, cmap="viridis")
ax.set(xlabel="x", ylabel="y")
ax.xaxis.tick_top()

In [ ]:
# Create a surface plot and projected filled contour plot under it.
fig = plt.figure()
ax = fig.add_subplot(projection="3d")

ax.plot_surface(
    X, Y, Z, rstride=3, cstride=3, linewidth=1, antialiased=True, cmap=cm.viridis
)

cset = ax.contourf(X, Y, Z, zdir="z", offset=-0.15, cmap=cm.viridis)

# Adjust the limits, ticks and view angle
fig
ax.set_zlim(-0.15, 0.2)
ax.set_zticks(np.linspace(0, 0.2, 5))
ax.view_init(27, -21)
# ax.view_init(0, 90)  # rotate view angle to see where mode(x) lies
# ax.view_init(0, 0)   # rotate view angle to see where mode(y) lies

plt.show()

### The covariance matrix is a real symmetric matrix.

Real symmetric matrices not only have real eigenvalues, they are always diagonalizable.

We can use its eigenvectors to diagonize it. In other words, we are looking for a linear combination of `x` and `y`, such that the new `x` and new `y` are decorrelated.

In [ ]:
from numpy.linalg import eig
eigenval, eigenvec = np.linalg.eig(sigma)

In [ ]:
sigma

In [ ]:
print(sigma_transformed := np.einsum('ij,jk,kl->il', eigenvec.T, sigma, eigenvec))

In [ ]:
# The matrix of eigenvectors is orthogonal, we can check this fact by the following.
print(np.dot(eigenvec.T, eigenvec), "\n")

# as well as...
print(np.dot(eigenvec, eigenvec.T))

In [ ]:
# The distribution on the variables X, Y packed into pos.
Z = multivariate_gaussian(pos, mu, sigma_transformed)

ax = sns.heatmap(Z, cmap="crest")
ax.set(xlabel="x", ylabel="y")
ax.xaxis.tick_top()